# 実行環境の作成

## GCP認証

下記コードでGCPに接続

In [1]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-japan
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-japan statistics-japan

## ライブラリ

In [ ]:
!pip install simplejson

# GBQからデータ取得

## テーブルデータ取得

In [3]:
from google.cloud import bigquery

def get_estat_origin(table_id):
  project_id='primal-buttress-342908'
  dataset_id = 'estat_ranking'
  client = bigquery.Client(project=project_id)

  query_string = f'SELECT * FROM {dataset_id}.{table_id}'

  return client.query(query_string).result().to_dataframe()

In [4]:
table_id = '0000010101_A1101'
get_estat_origin(table_id).head()

,index,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,45,A1101,01000,1975100000,5338206,人,総人口,北海道,1975年度,5
1,46,A1101,01000,1976100000,5395000,人,総人口,北海道,1976年度,5
2,47,A1101,01000,1977100000,5443000,人,総人口,北海道,1977年度,5
3,48,A1101,01000,1978100000,5490000,人,総人口,北海道,1978年度,5
4,49,A1101,01000,1979100000,5535000,人,総人口,北海道,1979年度,5


## テーブルデータを結合

In [39]:
card = {
    'cardId': 'total-population',
    'categories': [
        { '総数' : '0000010101_A1101' },
        { '男性' : '0000010101_A110101' },
        { '女性' : '0000010101_A110102' },
    ]
}

In [56]:
import pandas as pd

def df_data(card):
  
  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 結合
  for c in card['categories']:
    df = get_estat_origin(list(c.values())[0])
    df['categoryName'] = list(c.keys())[0]
    df['timeCode'] = df.apply(lambda x: x['timeCode'][:4], 1)
    df = df.drop('index', axis=1)

    df_res = pd.concat([df_res, df])
  
  return df_res

In [57]:
df_data(card).head()

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,A1101,01000,1975,5338206,人,総数,北海道,1975年度,5
1,A1101,01000,1976,5395000,人,総数,北海道,1976年度,5
2,A1101,01000,1977,5443000,人,総数,北海道,1977年度,5
3,A1101,01000,1978,5490000,人,総数,北海道,1978年度,5
4,A1101,01000,1979,5535000,人,総数,北海道,1979年度,5


## 地域ごとのJSONに保存

In [66]:
def save_json(card):

  df = df_data(card)
  
  areas = sorted(list(set(df['areaCode'].tolist())))
  for area in areas:
    df_area = df.copy().query(f'areaCode == "{area}"').reset_index()

    gcs_path = "statistics-japan/prefecture/{}_{}.json".format(card['cardId'],area)
    df_area.to_json(gcs_path, orient='records', force_ascii=False)
    print(f'{gcs_path}を作成しました')
    
  return

In [ ]:
save_json(card)

# 人口・世帯

### 総人口

In [ ]:
card = {
    'cardId': 'total-population',
    'categories': [
        { '総数' : '0000010101_A1101' },
        { '男性' : '0000010101_A110101' },
        { '女性' : '0000010101_A110102' },
    ]
}